每日收盤行情資訊
台灣證券交易所：https://www.twse.com.tw/zh/

In [ ]:
import requests
import pandas as pd
import sqlite3
import os


def get_daily_prices(date):
        
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX'
    
    payloads = {
        'response': 'html',
        'date': date,
        'type': 'ALLBUT0999'
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    }
    
    response = requests.get(url, headers=headers, params=payloads)
    
    try:
        df = pd.read_html(response.text)[-1]
    except:
        return None
    
    df.columns = df.columns.get_level_values(2)
    
    df.drop(['證券名稱', '漲跌(+/-)'], inplace=True, axis=1)
    
    df['日期'] = pd.to_datetime(date)
    
    df = df.set_index(['證券代號', '日期'])
    
    df = df.apply(pd.to_numeric, errors='coerce')
    
    df.drop(df[df['收盤價'].isnull()].index, inplace=True)
    
    return df

def save_daily_prices(new_df):
    
    connection = sqlite3.connect('data.db')
    
    try:
        df = pd.read_sql('select * from daily_prices', connection, parse_dates=['日期'], index_col=['證券代號', '日期'])
    except:
        df = pd.DataFrame()
    
    combined_df = df.append(new_df, sort=False) 
    
    final_df = combined_df.reset_index().drop_duplicates(subset=['證券代號', '日期'], keep='last').set_index(['證券代號', '日期']).sort_index()
    
    final_df.to_sql('daily_prices', connection, if_exists='replace')
    
    connection.close()
    
    excel_file = os.path.join('data',
                              'excel_files',
                              'daily_prices.xlsx')
    
    os.makedirs(os.path.dirname(excel_file), exist_ok=True)
    
    final_df.to_excel(excel_file)   

In [ ]:
import requests
import pandas as pd
import sqlite3
import os
import time


def retry_requests(url, headers, payloads):
    
    for i in range(3):
        try:
            return requests.get(url, headers=headers, params=payloads)
        except:
            print('發生錯誤，等待1分鐘後嘗試')
            time.sleep(60)
    
    return None

def get_daily_prices(date):
        
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX'
    
    payloads = {
        'response': 'html',
        'date': date,
        'type': 'ALLBUT0999'
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    }
    
    response = retry_requests(url, headers, payloads)
    
    try:
        df = pd.read_html(response.text)[-1]
    except:
        return None
    
    df.columns = df.columns.get_level_values(2)
    
    df.drop(['證券名稱', '漲跌(+/-)'], inplace=True, axis=1)
    
    df['日期'] = pd.to_datetime(date)
    
    df = df.set_index(['證券代號', '日期'])
    
    df = df.apply(pd.to_numeric, errors='coerce')
    
    df.drop(df[df['收盤價'].isnull()].index, inplace=True)
    
    return df

def save_daily_prices(new_df):
    
    connection = sqlite3.connect('data.db')
    
    try:
        df = pd.read_sql('select * from daily_prices', connection, parse_dates=['日期'], index_col=['證券代號', '日期'])
    except:
        df = pd.DataFrame()
    
    combined_df = df.append(new_df, sort=False) 
    
    final_df = combined_df.reset_index().drop_duplicates(subset=['證券代號', '日期'], keep='last').set_index(['證券代號', '日期']).sort_index()
    
    final_df.to_sql('daily_prices', connection, if_exists='replace')
    
    connection.close()
    
    excel_file = os.path.join('data',
                              'excel_files',
                              'daily_prices.xlsx')
    
    os.makedirs(os.path.dirname(excel_file), exist_ok=True)
    
    final_df.to_excel(excel_file)   

月報資訊
公開資訊觀測站：https://mops.twse.com.tw/mops/web/index

In [ ]:
import requests
import pandas as pd
import sqlite3
import os


def get_monthly_reports(date):
        
    url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_{year}_{month}_0.html'.format(year=date.year - 1911,
                                                                                      month=date.month)
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    }
    
    response = requests.get(url, headers=headers)
    response.encoding = 'big5'

    try:
        dfs = pd.read_html(response.text)
    except:
        return None
    
    df = pd.concat([df for df in dfs if len(df.columns) == 11])
    
    df.columns = df.columns.droplevel(0)
    
    df.drop(['公司名稱', '備註'], axis=1, inplace = True)
    
    df = df.rename(columns = {'公司代號':'證券代號'})

    df.reset_index(drop=True, inplace=True)
    df.drop(df[df['證券代號'] == '合計'].index, inplace=True)

    df['日期'] = pd.to_datetime(date)
    
    df = df.set_index(['證券代號', '日期'])
    
    return df

def save_monthly_reports(new_df):
    
    connection = sqlite3.connect('data.db')
    
    try:
        df = pd.read_sql('select * from monthly_reports', connection, parse_dates=['日期'], index_col=['證券代號', '日期'])
    except:
        df = pd.DataFrame()
    
    combined_df = df.append(new_df) 
    
    final_df = combined_df.reset_index().drop_duplicates(subset=['證券代號', '日期'], keep='last').set_index(['證券代號', '日期']).sort_index()
    
    final_df.to_sql('monthly_reports', connection, if_exists='replace')
    
    connection.close()
    
    excel_file = os.path.join('data',
                              'excel_files',
                              'monthly_reports.xlsx')
    
    os.makedirs(os.path.dirname(excel_file), exist_ok=True)
    
    final_df.to_excel(excel_file)   
    

In [ ]:
df = get_monthly_reports(pd.to_datetime('20200101'))
save_monthly_reports(df)
df